# Classes for data preparation and creating datasets

In [ ]:
#| default_exp datasets

In [ ]:
#| export
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import AutoTokenizer

In [ ]:
from en_grammar_checker.config import Config

In [ ]:
#| export
class BertClassificationDataset(Dataset):
    def __init__(
        self,
        cnfg,
        df: pd.DataFrame,
        is_test: bool = False,
        input_clm: str = "sentence",
        label_clm: str = "label",
    ):
        """
        cnfg: instance of Config class
        df: dataframe of data with label
        is_test: True if it for inference dataframe
        input_clm: column name for sentences
        label_clm: column name for label (dtype should not be object)
        """
        # get tokenizer from model name
        self.tokenizer = AutoTokenizer.from_pretrained(cnfg.base_model_name)
        self.df = df
        self.is_test = is_test
        self.cnfg = cnfg
        self.input_clm = input_clm
        self.label_clm = label_clm

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        row_tensors = []

        encoded_dict = self.tokenizer.encode_plus(
            row[self.input_clm],  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=self.cnfg.context_length,  # Pad & truncate all sentences.
            pad_to_max_length=True,
            return_attention_mask=True,  # Construct attn. masks.
            return_tensors="pt",  # Return pytorch tensors.
        )

        if self.is_test:
            return (
                encoded_dict["input_ids"].squeeze(),
                encoded_dict["attention_mask"].squeeze(),
            )
        else:
            label = torch.as_tensor(row[self.label_clm], dtype=torch.float32)
            return (
                encoded_dict["input_ids"].squeeze(),
                encoded_dict["attention_mask"].squeeze(),
                label,
            )

In [ ]:
#| export
def get_train_data_loader(
    cnfg,
    df,
    input_clm: str = "sentence",
    label_clm: str = "label",
):
    dataset = BertClassificationDataset(
        cnfg, df, is_test=False, input_clm=input_clm, label_clm=label_clm
    )
    dataloader = DataLoader(
        dataset,
        sampler=RandomSampler(dataset),  # Select batches randomly
        batch_size=cnfg.train_batch_size,
    )

    return dataloader

In [ ]:
#| export
def get_val_data_loader(
    cnfg,
    df,
    input_clm: str = "sentence",
    label_clm: str = "label",
):
    dataset = BertClassificationDataset(
        cnfg, df, is_test=False, input_clm=input_clm, label_clm=label_clm
    )
    dataloader = DataLoader(
        dataset,
        sampler=SequentialSampler(dataset),  # Select batches sequentialy
        batch_size=cnfg.val_batch_size,
    )

    return dataloader

In [ ]:
#| export
def get_test_data_loader(
    cnfg,
    df,
    input_clm: str = "sentence",
    label_clm=None,
):
    dataset = BertClassificationDataset(
        cnfg, df, is_test=True, input_clm=input_clm, label_clm=label_clm
    )
    dataloader = DataLoader(
        dataset,
        sampler=SequentialSampler(dataset),  # Select batches sequentialy
        batch_size=cnfg.test_batch_size,
    )

    return dataloader

### Testing DataSets

In [ ]:
cnfg = Config()

In [ ]:
df_train = pd.read_csv(
    f"{cnfg.train_path}",
    delimiter="\t",
    header=None,
    names=["sentence_source", "label", "label_notes", "sentence"],
)

In [ ]:
train_dataloader = get_train_data_loader(cnfg, df_train)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
train_dataloader_iterator = iter(train_dataloader)
X, X2, Y = next(train_dataloader_iterator)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
X.shape, X2.shape, Y.shape

(torch.Size([32, 512]), torch.Size([32, 512]), torch.Size([32]))

In [ ]:
X

tensor([[   1,  585, 1234,  ...,    0,    0,    0],
        [   1,  273,  481,  ...,    0,    0,    0],
        [   1,  512,  313,  ...,    0,    0,    0],
        ...,
        [   1,  273, 1659,  ...,    0,    0,    0],
        [   1,  918, 3721,  ...,    0,    0,    0],
        [   1, 1887,  261,  ...,    0,    0,    0]])

In [ ]:
X2

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])